In [1]:
#some libraries
!pip install -qU \
  transformers==4.31.0 \
  sentence-transformers==2.2.2 \
  pinecone-client==3.1.0 \
  accelerate==0.21.0 \
  einops==0.6.1 \
  langchain==0.0.240 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.8 MB/s eta 0:00:00
  

In [10]:
#specific version to avoid error with vector store
#!pip install chromadb==0.5.3
#!pip install datasets

In [3]:
#loading the llama-2-7b-chat model and the tokenizer
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, an authentication token is needed
hf_auth = 'hugging_face_token'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")
#tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [4]:
#mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#loading training data for providing context
import pandas as pd
train_data_df=pd.read_csv('/content/drive/MyDrive/train_data.csv')
## Convert the DataFrame to a format compatible with Hugging Face Datasets
train_data_df['Instruction'] = train_data_df['Instruction']+ " Response:\n\n"+ train_data_df['Output'].astype(str)
train_dataset_dict = {
    'instruction': train_data_df['Instruction'].tolist()
}

In [6]:
#creating documents from the list
docs=train_dataset_dict['instruction']

In [7]:
#initializing the embedding model
from sentence_transformers import SentenceTransformer
import chromadb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
import torch
embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', cache_folder = '/data/base_models')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [8]:
#do the embedding
embeddings = embedding_model.encode(docs)
#verify the embeddings
embeddings.shape

(1040, 768)

In [9]:
#indexing-run this only once
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="q-rag_llama2")

In [11]:
#addding the train data
collection.add(
    embeddings = embeddings,
    documents=docs,
    ids= [str(i) for i in range(len(docs))]
)

In [12]:
#retrieving data from the query store
def retrieve_vector_db(query, n_results=3):
    results = collection.query(
    query_embeddings = embedding_model.encode(query).tolist(),
    n_results=n_results
    )
    return results['documents']

In [13]:
#Generating some response with llama 2 model
def get_llama2_chat_reponse(prompt, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, temperature= 0.00001)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [25]:
#sample query
#we can have different queries depending on the property we are interested in
query = "Please extract the design type of the quantum cascade laser device. Please print none if the value is not in text and do not give any explanations. In the output, just include only the extracted property."
retrieved_results = retrieve_vector_db(query)

In [26]:
#viewing the results
retrieved_results[0]

['From the following sentence, please extract the design type of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text: Utilizing an innovative scattering-driven injection and resonant-phonon depleting technique, a pioneering quantum cascade laser with a frequency of 9.2 THz was successfully produced. Response:\n\nresonant-phonon',
 'From the following sentence, please extract the design type of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text:Through the innovative implementation of a scattering-assisted injection and resonant-phonon depopulation approach, scientists have achieved a new milestone with a 13.5 THz quantum cascade laser. Response:\n\nresonant-phonon',
 'From the following sentence, please extract the design type of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text: A revolutionary approach involving scattering-a

In [27]:
#assigning the context
context=retrieved_results[0]

In [28]:
#user input-entails a string of text that contains the properties
#input_text= input("Enter the text here: ")
input_text= "2.1 THz quantum cascade laser (QCL) based on a scatterin gassisted injection and resonant-phonon depopulation design scheme is demonstrated. The QCL is based on a four-well period implemented in theGaAs/Al0:15Ga0:85As material system. The QCL operates up to a heat-sink temperature of 144 K in pulsed-mode, which is considerably higher than that achieved for previously reported THz QCLs operating around the frequency of 2 THz. At 46 K, the threshold current-density was measured as \~ 745 A/cm2 with a peak-power output of \~ 10 mW. Electrically stable operation in a positive differential-resistance regime is achieved by a careful choice of design parameters. The results validate the robustness of scattering-assisted injection schemes for development of low-frequency (n < 2:5 THz) QCLs."
#print(input_text)

In [29]:
#prompt template
prompt = f'''
[INST]
Problem Definition: Extraction of quantum cascade laser properties from text entails extracting properties from a given text description. This should be done without providing any other additional information or explanations.

Example Sentences containing instructions for extracting properties, the input text and the corresponding extracted properties: {context}

Instruction: {query}

Input Sentence: {input_text}

[/INST]
'''

In [30]:
print(prompt)


[INST]
Problem Definition: Extraction of quantum cascade laser properties from text entails extracting properties from a given text description. This should be done without providing any other additional information or explanations.

Example Sentences containing instructions for extracting properties, the input text and the corresponding extracted properties: ['From the following sentence, please extract the design type of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text: Utilizing an innovative scattering-driven injection and resonant-phonon depleting technique, a pioneering quantum cascade laser with a frequency of 9.2 THz was successfully produced. Response:\n\nresonant-phonon', 'From the following sentence, please extract the design type of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text:Through the innovative implementation of a scattering-assisted injection and resona

In [31]:
print(get_llama2_chat_reponse(prompt, max_new_tokens=800))


[INST]
Problem Definition: Extraction of quantum cascade laser properties from text entails extracting properties from a given text description. This should be done without providing any other additional information or explanations.

Example Sentences containing instructions for extracting properties, the input text and the corresponding extracted properties: ['From the following sentence, please extract the design type of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text: Utilizing an innovative scattering-driven injection and resonant-phonon depleting technique, a pioneering quantum cascade laser with a frequency of 9.2 THz was successfully produced. Response:\n\nresonant-phonon', 'From the following sentence, please extract the design type of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text:Through the innovative implementation of a scattering-assisted injection and resona

In [32]:
#get some output with transformers pipeline
#import pipeline
from transformers import pipeline

In [33]:
#Using Pipeline from the hugging face
instruction = prompt
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=800)
result = pipe(instruction)
#Trim the response, remove instruction manually
print(result[0]['generated_text'][len(instruction):])

Sure, I'd be happy to help you extract the design type of the quantum cascade laser device from the given text. Based on the input sentence you provided, the extracted property is:

Resonant-phonon
